In [1]:
!pip install -q werkzeug

In [2]:
import os
import json
#import random
import string
#import base64
#import hashlib
import numpy as np
import pandas as pd
from werkzeug import security
from google.colab import drive
#from google.cloud import firestore
import firebase_admin
from firebase_admin import firestore, auth, credentials

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#JSON_KEY_FILE = '/content/drive/My Drive/firestore-446822-ac05de749f4a.json'

#JSON_KEY_FILE = "/content/drive/My Drive/service-pa-79ac5-0f0babe13506.json"

JSON_KEY_FILE = "/content/drive/My Drive/csg3101-service-providing-app-firebase-adminsdk-7xpfw-f39bc603db.json"

In [5]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = JSON_KEY_FILE

# Initialize Firebase Admin SDK
cred = credentials.Certificate(JSON_KEY_FILE)
firebase_admin.initialize_app(cred)

In [6]:
JSON_DATA_FILE = '/content/drive/My Drive/yelp_academic_dataset_business.json'

In [7]:
# Define the number of lines to read
#n_lines = 1

# Read the specified number of lines into a list of dictionaries
#with open(JSON_DATA_FILE, "r") as file:
#    data = [json.loads(next(file)) for _ in range(n_lines)]

# Read the JSON lines file directly into a pandas DataFrame
df = pd.read_json(JSON_DATA_FILE, lines=True)

# Convert the list of dictionaries into a DataFrame
#df = pd.DataFrame(data)

# Display the first few rows
print(df.head())

              business_id                      name  \
0  Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2  tUFrWirKiKi_TAnsVWINQQ                    Target   
3  MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4  mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   

                           address           city state postal_code  \
0           1616 Chapala St, Ste 2  Santa Barbara    CA       93101   
1  87 Grasso Plaza Shopping Center         Affton    MO       63123   
2             5255 E Broadway Blvd         Tucson    AZ       85711   
3                      935 Race St   Philadelphia    PA       19107   
4                    101 Walnut St     Green Lane    PA       18054   

    latitude   longitude  stars  review_count  is_open  \
0  34.426679 -119.711197    5.0             7        0   
1  38.551126  -90.335695    3.0            15        1   
2  32.223236 -110.880452    3.5            22        0   
3  39.9555

In [8]:
print(len(df)) # total number of entries

150346


In [9]:
# Filter rows where 'categories' is not null
df = df[df['categories'].notnull()]

# Select specific columns
df = df[['business_id', 'name', 'address', 'latitude', 'longitude', 'stars', 'review_count', 'categories']]

# Display the result
print(df.head())

              business_id                      name  \
0  Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2  tUFrWirKiKi_TAnsVWINQQ                    Target   
3  MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4  mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   

                           address   latitude   longitude  stars  \
0           1616 Chapala St, Ste 2  34.426679 -119.711197    5.0   
1  87 Grasso Plaza Shopping Center  38.551126  -90.335695    3.0   
2             5255 E Broadway Blvd  32.223236 -110.880452    3.5   
3                      935 Race St  39.955505  -75.155564    4.0   
4                    101 Walnut St  40.338183  -75.471659    4.5   

   review_count                                         categories  
0             7  Doctors, Traditional Chinese Medicine, Naturop...  
1            15  Shipping Centers, Local Services, Notaries, Ma...  
2            22  Department Stores, Shopping, Fashion

In [10]:
print(len(df))  # number of entries after removing 103 rows where 'categories' have null value

150243


In [11]:
# Split 'categories' into a list of categories
df['categories'] = df['categories'].str.split(', ')

# Rename columns
df = df.rename(columns={'business_id': 'id', 'name': 'firstName', 'review_count': 'numOfReviews', 'categories': 'skills'})

# Display the result
print(df.head())

                       id                 firstName  \
0  Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2  tUFrWirKiKi_TAnsVWINQQ                    Target   
3  MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4  mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   

                           address   latitude   longitude  stars  \
0           1616 Chapala St, Ste 2  34.426679 -119.711197    5.0   
1  87 Grasso Plaza Shopping Center  38.551126  -90.335695    3.0   
2             5255 E Broadway Blvd  32.223236 -110.880452    3.5   
3                      935 Race St  39.955505  -75.155564    4.0   
4                    101 Walnut St  40.338183  -75.471659    4.5   

   numOfReviews                                             skills  
0             7  [Doctors, Traditional Chinese Medicine, Naturo...  
1            15  [Shipping Centers, Local Services, Notaries, M...  
2            22  [Department Stores, Shopping, Fashio

In [12]:
# Function to generate a random phone number
def generate_random_phone():
    # Random phone number in the format XXX-XXX-XXXX
    mobile_operator_code = "0" + str(np.random.randint(70, 78))
    first_part = np.random.randint(100, 999)
    second_part = np.random.randint(1000, 9999)
    return f"{mobile_operator_code}-{first_part}-{second_part}"

In [13]:
# Function to generate a random password
def generate_random_password(length=12):
    # Generate a random password with letters, digits, and special characters
    characters = string.ascii_letters + string.digits + string.punctuation
    #return ''.join(random.choice(characters) for _ in range(length))
    return ''.join(np.random.choice(list(characters), size=length))

In [14]:
#df["role"] = "employee" # All rows have the same value for role

df["status"] = "active"  # All rows have the same value for status

# Display the result
print(df.head())

                       id                 firstName  \
0  Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2  tUFrWirKiKi_TAnsVWINQQ                    Target   
3  MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4  mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   

                           address   latitude   longitude  stars  \
0           1616 Chapala St, Ste 2  34.426679 -119.711197    5.0   
1  87 Grasso Plaza Shopping Center  38.551126  -90.335695    3.0   
2             5255 E Broadway Blvd  32.223236 -110.880452    3.5   
3                      935 Race St  39.955505  -75.155564    4.0   
4                    101 Walnut St  40.338183  -75.471659    4.5   

   numOfReviews                                             skills  status  
0             7  [Doctors, Traditional Chinese Medicine, Naturo...  active  
1            15  [Shipping Centers, Local Services, Notaries, M...  active  
2            22  [Department 

In [15]:
# Add a new empty 'last_name' column
df['lastName'] = ""

# Display the result
print(df.head())

                       id                 firstName  \
0  Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2  tUFrWirKiKi_TAnsVWINQQ                    Target   
3  MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4  mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   

                           address   latitude   longitude  stars  \
0           1616 Chapala St, Ste 2  34.426679 -119.711197    5.0   
1  87 Grasso Plaza Shopping Center  38.551126  -90.335695    3.0   
2             5255 E Broadway Blvd  32.223236 -110.880452    3.5   
3                      935 Race St  39.955505  -75.155564    4.0   
4                    101 Walnut St  40.338183  -75.471659    4.5   

   numOfReviews                                             skills  status  \
0             7  [Doctors, Traditional Chinese Medicine, Naturo...  active   
1            15  [Shipping Centers, Local Services, Notaries, M...  active   
2            22  [Departme

In [16]:
# Add a new empty 'profile_picture_url' column
df['profileImageUrl'] = ""

# Display the result
print(df.head())

                       id                 firstName  \
0  Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2  tUFrWirKiKi_TAnsVWINQQ                    Target   
3  MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4  mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   

                           address   latitude   longitude  stars  \
0           1616 Chapala St, Ste 2  34.426679 -119.711197    5.0   
1  87 Grasso Plaza Shopping Center  38.551126  -90.335695    3.0   
2             5255 E Broadway Blvd  32.223236 -110.880452    3.5   
3                      935 Race St  39.955505  -75.155564    4.0   
4                    101 Walnut St  40.338183  -75.471659    4.5   

   numOfReviews                                             skills  status  \
0             7  [Doctors, Traditional Chinese Medicine, Naturo...  active   
1            15  [Shipping Centers, Local Services, Notaries, M...  active   
2            22  [Departme

In [17]:
# Add a new 'display_name' column
#df['display_name'] = df["first_name"]

# Display the result
#print(df.head())

In [18]:
# Add a new empty 'certificate_url' column
df['certificateUrl'] = [[]] * len(df)

# Display the result
print(df.head())

                       id                 firstName  \
0  Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2  tUFrWirKiKi_TAnsVWINQQ                    Target   
3  MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4  mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   

                           address   latitude   longitude  stars  \
0           1616 Chapala St, Ste 2  34.426679 -119.711197    5.0   
1  87 Grasso Plaza Shopping Center  38.551126  -90.335695    3.0   
2             5255 E Broadway Blvd  32.223236 -110.880452    3.5   
3                      935 Race St  39.955505  -75.155564    4.0   
4                    101 Walnut St  40.338183  -75.471659    4.5   

   numOfReviews                                             skills  status  \
0             7  [Doctors, Traditional Chinese Medicine, Naturo...  active   
1            15  [Shipping Centers, Local Services, Notaries, M...  active   
2            22  [Departme

In [19]:
df["email"] = df["id"].str.lower() + "@employee.com" # Email based on id

# Display the result
print(df.head())

                       id                 firstName  \
0  Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2  tUFrWirKiKi_TAnsVWINQQ                    Target   
3  MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4  mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   

                           address   latitude   longitude  stars  \
0           1616 Chapala St, Ste 2  34.426679 -119.711197    5.0   
1  87 Grasso Plaza Shopping Center  38.551126  -90.335695    3.0   
2             5255 E Broadway Blvd  32.223236 -110.880452    3.5   
3                      935 Race St  39.955505  -75.155564    4.0   
4                    101 Walnut St  40.338183  -75.471659    4.5   

   numOfReviews                                             skills  status  \
0             7  [Doctors, Traditional Chinese Medicine, Naturo...  active   
1            15  [Shipping Centers, Local Services, Notaries, M...  active   
2            22  [Departme

In [20]:
# Add a new 'timestamp' column
df['dateOfJoin'] = firestore.SERVER_TIMESTAMP

# Display the result
print(df.head())

                       id                 firstName  \
0  Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2  tUFrWirKiKi_TAnsVWINQQ                    Target   
3  MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4  mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   

                           address   latitude   longitude  stars  \
0           1616 Chapala St, Ste 2  34.426679 -119.711197    5.0   
1  87 Grasso Plaza Shopping Center  38.551126  -90.335695    3.0   
2             5255 E Broadway Blvd  32.223236 -110.880452    3.5   
3                      935 Race St  39.955505  -75.155564    4.0   
4                    101 Walnut St  40.338183  -75.471659    4.5   

   numOfReviews                                             skills  status  \
0             7  [Doctors, Traditional Chinese Medicine, Naturo...  active   
1            15  [Shipping Centers, Local Services, Notaries, M...  active   
2            22  [Departme

In [21]:
# Read the JSON file containing star category counts into a DataFrame
star_category_counts = pd.read_json('/content/drive/My Drive/Employee_Star_Categories.json', orient='records', lines=True)

In [22]:
print(len(star_category_counts)) # number of entries

150346


In [23]:
# Display the first few rows in 'star_category_counts' dataframe
print(star_category_counts.head())

                       id                                  starCategoryCount
0  ---kPU91CF4Lq2-WlRu9Lw  {'5 stars': 17, '4 stars': 4, '3 stars': 2, '2...
1  --0iUa4sNDFiZFrAdIWhZQ  {'5 stars': 2, '4 stars': 5, '3 stars': 4, '2 ...
2  --30_8IhuyMHbSOcNWd6DQ  {'5 stars': 5, '4 stars': 1, '3 stars': 0, '2 ...
3  --7PUidqRWpRSpXebiyxTg  {'5 stars': 0, '4 stars': 1, '3 stars': 2, '2 ...
4  --7jw19RH9JKXgFohspgQw  {'5 stars': 10, '4 stars': 0, '3 stars': 1, '2...


In [24]:
# Merge the two DataFrames based on 'id', keeping only matching 'id' rows
df = pd.merge(df, star_category_counts[['id', 'starCategoryCount']], on='id', how='left')

# Display the result
print(df.head())

                       id                 firstName  \
0  Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2  tUFrWirKiKi_TAnsVWINQQ                    Target   
3  MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4  mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   

                           address   latitude   longitude  stars  \
0           1616 Chapala St, Ste 2  34.426679 -119.711197    5.0   
1  87 Grasso Plaza Shopping Center  38.551126  -90.335695    3.0   
2             5255 E Broadway Blvd  32.223236 -110.880452    3.5   
3                      935 Race St  39.955505  -75.155564    4.0   
4                    101 Walnut St  40.338183  -75.471659    4.5   

   numOfReviews                                             skills  status  \
0             7  [Doctors, Traditional Chinese Medicine, Naturo...  active   
1            15  [Shipping Centers, Local Services, Notaries, M...  active   
2            22  [Departme

In [25]:
# Add a new 'experience' column experience between 0 and 20 (0 and 20 inclusive)
df['experience'] = np.random.randint(0, 21, size=len(df))

# Display the result
print(df.head())

                       id                 firstName  \
0  Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2  tUFrWirKiKi_TAnsVWINQQ                    Target   
3  MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4  mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   

                           address   latitude   longitude  stars  \
0           1616 Chapala St, Ste 2  34.426679 -119.711197    5.0   
1  87 Grasso Plaza Shopping Center  38.551126  -90.335695    3.0   
2             5255 E Broadway Blvd  32.223236 -110.880452    3.5   
3                      935 Race St  39.955505  -75.155564    4.0   
4                    101 Walnut St  40.338183  -75.471659    4.5   

   numOfReviews                                             skills  status  \
0             7  [Doctors, Traditional Chinese Medicine, Naturo...  active   
1            15  [Shipping Centers, Local Services, Notaries, M...  active   
2            22  [Departme

In [26]:
# Add a new 'nic' column (12 digits)
df['nic'] = np.random.randint(10**11, 10**12, size=len(df))

# Display the result
print(df.head())

                       id                 firstName  \
0  Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2  tUFrWirKiKi_TAnsVWINQQ                    Target   
3  MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4  mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   

                           address   latitude   longitude  stars  \
0           1616 Chapala St, Ste 2  34.426679 -119.711197    5.0   
1  87 Grasso Plaza Shopping Center  38.551126  -90.335695    3.0   
2             5255 E Broadway Blvd  32.223236 -110.880452    3.5   
3                      935 Race St  39.955505  -75.155564    4.0   
4                    101 Walnut St  40.338183  -75.471659    4.5   

   numOfReviews                                             skills  status  \
0             7  [Doctors, Traditional Chinese Medicine, Naturo...  active   
1            15  [Shipping Centers, Local Services, Notaries, M...  active   
2            22  [Departme

In [27]:
# Generate a random employee charge per hour between 2500 and 3000
df["chargePerHour"] = np.random.randint(2500, 3000, size=len(df))  # Random employee charge per hour for each row

# Display the result
print(df.head())

                       id                 firstName  \
0  Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2  tUFrWirKiKi_TAnsVWINQQ                    Target   
3  MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4  mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   

                           address   latitude   longitude  stars  \
0           1616 Chapala St, Ste 2  34.426679 -119.711197    5.0   
1  87 Grasso Plaza Shopping Center  38.551126  -90.335695    3.0   
2             5255 E Broadway Blvd  32.223236 -110.880452    3.5   
3                      935 Race St  39.955505  -75.155564    4.0   
4                    101 Walnut St  40.338183  -75.471659    4.5   

   numOfReviews                                             skills  status  \
0             7  [Doctors, Traditional Chinese Medicine, Naturo...  active   
1            15  [Shipping Centers, Local Services, Notaries, M...  active   
2            22  [Departme

In [28]:
#df["location"] = firestore.GeoPoint(float(df["latitude"]), float(df["longitude"])) # Create a GeoPoint using the latitude and longitude

df['location'] = df.apply(lambda row: firestore.GeoPoint(float(row['latitude']), float(row['longitude'])), axis=1) # Create a GeoPoint using the latitude and longitude

# Display the result
print(df.head())

                       id                 firstName  \
0  Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2  tUFrWirKiKi_TAnsVWINQQ                    Target   
3  MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4  mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   

                           address   latitude   longitude  stars  \
0           1616 Chapala St, Ste 2  34.426679 -119.711197    5.0   
1  87 Grasso Plaza Shopping Center  38.551126  -90.335695    3.0   
2             5255 E Broadway Blvd  32.223236 -110.880452    3.5   
3                      935 Race St  39.955505  -75.155564    4.0   
4                    101 Walnut St  40.338183  -75.471659    4.5   

   numOfReviews                                             skills  status  \
0             7  [Doctors, Traditional Chinese Medicine, Naturo...  active   
1            15  [Shipping Centers, Local Services, Notaries, M...  active   
2            22  [Departme

In [29]:
df = df.drop(columns=['latitude', 'longitude']) # Remove the 'latitude' and 'longitude' columns from the DataFrame

# Display the result
print(df.head())

                       id                 firstName  \
0  Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2  tUFrWirKiKi_TAnsVWINQQ                    Target   
3  MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4  mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   

                           address  stars  numOfReviews  \
0           1616 Chapala St, Ste 2    5.0             7   
1  87 Grasso Plaza Shopping Center    3.0            15   
2             5255 E Broadway Blvd    3.5            22   
3                      935 Race St    4.0            80   
4                    101 Walnut St    4.5            13   

                                              skills  status lastName  \
0  [Doctors, Traditional Chinese Medicine, Naturo...  active            
1  [Shipping Centers, Local Services, Notaries, M...  active            
2  [Department Stores, Shopping, Fashion, Home & ...  active            
3  [Restaurants, Food,

In [30]:
df["phone"] = [generate_random_phone() for _ in range(len(df))]  # Random phone number for each row

# Display the result
print(df.head())

                       id                 firstName  \
0  Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2  tUFrWirKiKi_TAnsVWINQQ                    Target   
3  MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4  mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   

                           address  stars  numOfReviews  \
0           1616 Chapala St, Ste 2    5.0             7   
1  87 Grasso Plaza Shopping Center    3.0            15   
2             5255 E Broadway Blvd    3.5            22   
3                      935 Race St    4.0            80   
4                    101 Walnut St    4.5            13   

                                              skills  status lastName  \
0  [Doctors, Traditional Chinese Medicine, Naturo...  active            
1  [Shipping Centers, Local Services, Notaries, M...  active            
2  [Department Stores, Shopping, Fashion, Home & ...  active            
3  [Restaurants, Food,

In [31]:
df["password"] = [generate_random_password() for _ in range(len(df))]  # Random password for each row

# Display the result
print(df.head())

                       id                 firstName  \
0  Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2  tUFrWirKiKi_TAnsVWINQQ                    Target   
3  MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4  mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   

                           address  stars  numOfReviews  \
0           1616 Chapala St, Ste 2    5.0             7   
1  87 Grasso Plaza Shopping Center    3.0            15   
2             5255 E Broadway Blvd    3.5            22   
3                      935 Race St    4.0            80   
4                    101 Walnut St    4.5            13   

                                              skills  status lastName  \
0  [Doctors, Traditional Chinese Medicine, Naturo...  active            
1  [Shipping Centers, Local Services, Notaries, M...  active            
2  [Department Stores, Shopping, Fashion, Home & ...  active            
3  [Restaurants, Food,

In [32]:
#df["hashedPassword"] = df["password"].apply(lambda x: security.generate_password_hash(x))

# Display the result
#print(df.head())

In [33]:
# Convert to NumPy array and hash passwords
#df["hashedPassword"] = np.vectorize(security.generate_password_hash)(df["password"].values)

# Display the result
#print(df.head())

In [34]:
# Efficiently hash passwords using a list comprehension
#df["hashedPassword"] = [security.generate_password_hash(password) for password in df["password"]]

# Display the result
#print(df.head())

In [35]:
# Display the result
#print(df.head())

In [36]:
# Copy 'id' and 'password' columns to a new DataFrame
new_df = df[['id', 'email', 'password']].copy()

# Save the new DataFrame to a local JSON file
new_df.to_json("/content/drive/My Drive/employee_passwords.json", orient="records", lines=True)

# Display the result
print(new_df.head())

                       id                                email      password
0  Pns2l4eNsfO8kk83dixA6A  pns2l4ensfo8kk83dixa6a@employee.com  {z"5`qU&^YUV
1  mpf3x-BjTdTEA3yCZrAYPw  mpf3x-bjtdtea3yczraypw@employee.com  h2fr:wA;vE*_
2  tUFrWirKiKi_TAnsVWINQQ  tufrwirkiki_tansvwinqq@employee.com  CRK_v~#-*at|
3  MTSW4McQd7CbVtyjqoe9mw  mtsw4mcqd7cbvtyjqoe9mw@employee.com  ZL{)jbbMGWl:
4  mWMc6_wTdE0EUBKIGXDVfA  mwmc6_wtde0eubkigxdvfa@employee.com  v0`g6rD&7::*


In [37]:
# Remove 'password' column from the original DataFrame
#df = df.drop(columns=['password'])

# Rename column
#df = df.rename(columns={'hashedPassword': 'password'})

# Display the result
#print(df.head())

In [38]:
# Convert DataFrame to JSON
processed_data = df.to_dict(orient='records')

In [39]:
print(processed_data[0])

{'id': 'Pns2l4eNsfO8kk83dixA6A', 'firstName': 'Abby Rappoport, LAC, CMQ', 'address': '1616 Chapala St, Ste 2', 'stars': 5.0, 'numOfReviews': 7, 'skills': ['Doctors', 'Traditional Chinese Medicine', 'Naturopathic/Holistic', 'Acupuncture', 'Health & Medical', 'Nutritionists'], 'status': 'active', 'lastName': '', 'profileImageUrl': '', 'certificateUrl': [], 'email': 'pns2l4ensfo8kk83dixa6a@employee.com', 'dateOfJoin': Sentinel: Value used to set a document field to the server timestamp., 'starCategoryCount': {'5 stars': 6, '4 stars': 1, '3 stars': 0, '2 stars': 0, '1 stars': 0}, 'experience': 0, 'nic': 393723962528, 'chargePerHour': 2877, 'location': <google.cloud.firestore_v1._helpers.GeoPoint object at 0x7c5ca94ac290>, 'phone': '076-279-3770', 'password': '{z"5`qU&^YUV'}


In [40]:
print(len(processed_data)) # num of entries

150243


In [41]:
# Initialize Firestore client
db = firestore.Client()

In [42]:
# Firestore collection name
collection_name = "employees"

In [43]:
# Upload processed data to Firestore
for i, doc in enumerate(processed_data, start=1):

    # Extract the 'id' before removing it from the document body
    doc_id = doc.pop('id', None)

    # Upload the document without the 'id' in the body
    doc_ref = db.collection(collection_name).document(doc_id)
    doc_ref.set(doc)

    # Print message every 10,000 iterations
    if i % 10000 == 0:
      print(f"{i} employees uploaded.")

10000 employees uploaded.
20000 employees uploaded.
30000 employees uploaded.
40000 employees uploaded.
50000 employees uploaded.
60000 employees uploaded.
70000 employees uploaded.
80000 employees uploaded.
90000 employees uploaded.
100000 employees uploaded.
110000 employees uploaded.
120000 employees uploaded.
130000 employees uploaded.
140000 employees uploaded.
150000 employees uploaded.


In [44]:
# Upload processed data to Firestore
#for doc in processed_data:

  # Extract the 'id' before removing it from the document body
#  doc_id = doc.pop('id', None)
#  doc_password = doc.pop('password', None)


  # Create a user in Firebase Authentication with a custom UID
#  user = auth.create_user(
#      uid=doc_id,  # Custom UID
#      email=doc['email'],
#      password=doc_password
#      )

  # Upload the document without the 'id' in the body
#  doc_ref = db.collection(collection_name).document(doc_id)
#  doc_ref.set(doc)

In [45]:
print("Processed data uploaded successfully!")

Processed data uploaded successfully!
